In [48]:
import os
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker import get_execution_role

from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel

from time import gmtime, strftime

In [49]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_boto_client = boto_session.client("sagemaker")

# sagemaker_session = sagemaker.session.Session()
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)

sm_role = get_execution_role()

In [50]:
models = sm_client.list_models()
# models

model_name = models['Models'][0]['ModelName']
# model_name

response = sm_client.describe_model(ModelName=model_name)
# response

model_data_url = response['PrimaryContainer']['ModelDataUrl']
model_data_url

's3://artwork-content-trial-bucket/pipeline-data/pipelines-n8smdtmxtmty-TrainingStep-URxRcTDFSN/output/model.tar.gz'

In [51]:
 # Create a model
BASE_DIR = os.path.realpath('.')
entry_script='inference.py'
source_dir = os.path.join(BASE_DIR, 'scoring')

model = SKLearnModel(
    role=sm_role,
    model_data=model_data_url,
    framework_version="1.0-1",
    py_version="py3",
    source_dir=source_dir,
    entry_point=entry_script,
)

In [52]:
endpoint_name = 'mch-content-ep-2'
model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name=endpoint_name)

-----!

In [53]:
# pipeline.delete()

In [54]:
import boto3
client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {
    "itemId": "143571",
    "userId": "650334e4-9f4b-ed11-bba2-000d3ad98c7a",
    "performedAction": "ENTER_ARTWORK_DETAIL"
}
endpoint_name = endpoint_name

print('Input Data: ', request_body)

import json
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

# response
result = json.loads(response['Body'].read().decode())
print('\n Output: ', result)

Input Data:  {'itemId': '143571', 'userId': '650334e4-9f4b-ed11-bba2-000d3ad98c7a', 'performedAction': 'ENTER_ARTWORK_DETAIL'}

 Output:  {'rec': [143518, 143519, 143520, 143470, 150512, 147955, 146236, 145703, 146199, 146039]}
